In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python.keras.callbacks import TensorBoard
from time import time
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
full_df = pd.read_csv("../resources/full_dataset.csv")

In [3]:
def generate_spectrogram(full_df, spec_shape, fs=200, window=('tukey', 0.25), nperseg=50, noverlap=None):
    full_df = full_df.sample(frac=1).reset_index(drop=True)
    labels = full_df["labels"]
    features = full_df.drop(labels="labels", axis='columns')
    features = features.values.reshape(-1, 400, 4)
    arr = np.zeros((1000, spec_shape[0], spec_shape[1], 4))
    first = True
    for i in range(1000):
        _, _, spec_1 = signal.spectrogram(features[i][:, 0], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_1 = spec_1.reshape(spec_shape[0], spec_shape[1], 1)
        _, _, spec_2 = signal.spectrogram(features[i][:, 1], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_2 = spec_2.reshape(spec_shape[0], spec_shape[1], 1)
        _, _, spec_3 = signal.spectrogram(features[i][:, 2], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_3 = spec_3.reshape(spec_shape[0], spec_shape[1], 1)
        _, _, spec_4 = signal.spectrogram(features[i][:, 3], fs=fs, window=window,
                                               nperseg=nperseg, noverlap=noverlap)
        spec_4 = spec_4.reshape(spec_shape[0], spec_shape[1], 1)
        if first:
            print("Spectrogram per channel shape {}".format(spec_1.shape))
            first = False
        arr[i] = np.concatenate((spec_1, spec_2, spec_3, spec_4), axis=2)
    print("arr shape {} --> {}".format(arr.shape, arr.reshape(-1, spec_shape[0] * spec_shape[1] * 4).shape))
    arr = arr.reshape(-1, spec_shape[0] * spec_shape[1] * 4)
    columns = []
    for i in range(spec_shape[0] * spec_shape[1] * 4):
        columns.append("p{}".format(i))
    new_df = pd.DataFrame(data=arr, columns=columns)
    new_df["labels"] = labels
    return (new_df)

In [4]:
data = generate_spectrogram(full_df, spec_shape=(26, 8))

Spectrogram per channel shape (26, 8, 1)
arr shape (1000, 26, 8, 4) --> (1000, 832)


In [5]:
def preprocess_data(data, spec_shape):
    labels = data["labels"].values
    labels = np.where(labels == "Lights-on", 1, 0) # 1 for Lights-on and 0 for Turn-off
    labels = tf.keras.utils.to_categorical(labels)
    data = data.drop(labels="labels", axis='columns')
    data = data.values.reshape(-1, spec_shape[0], spec_shape[1], 4)
    return (data, labels)

In [6]:
test = data.head(200)
training = data.tail(800)

In [7]:
features, labels = preprocess_data(training, spec_shape=(26, 8))

In [22]:
tensorboard = TensorBoard(log_dir="logs/model_1_{}".format(time()))

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, strides=(1, 1), padding="same",
                                 activation='relu', kernel_initializer='he_normal', input_shape=(26, 8, 4)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, strides=(1, 1),
                                 padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"))
# model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(1, 1),
#                                  padding="same", activation='relu', kernel_initializer='he_normal'))
# model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(1, 1),
#                                  padding="same", activation='relu', kernel_initializer='he_normal'))
# model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(1, 1),
#                                  padding="same", activation='relu', kernel_initializer='he_normal'))
# model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), padding="same"))
# model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=(1, 1),
#                                  padding="same", activation='relu', kernel_initializer='he_normal'))
# model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=(1, 1),
#                                  padding="same", activation='relu', kernel_initializer='he_normal'))
# model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=(1, 1),
#                                  padding="same", activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 26, 8, 32)         544       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 26, 8, 32)         4128      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 13, 4, 32)         0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 4, 64)         8256      
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 13, 4, 64)         16448     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 2, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 896)              

In [23]:
model.fit(x=features,
          y=labels,
          batch_size=16,
          epochs=10,
          validation_split=0.1,
          callbacks=[tensorboard])

Train on 720 samples, validate on 80 samples
Epoch 1/10
720/720 [==============================] - 1s 1ms/sample - loss: 7.7361 - accuracy: 0.4931 - val_loss: 5.1805 - val_accuracy: 0.5250
Epoch 2/10
720/720 [==============================] - 0s 619us/sample - loss: 2.4501 - accuracy: 0.7583 - val_loss: 1.1241 - val_accuracy: 0.8625
Epoch 3/10
720/720 [==============================] - 0s 609us/sample - loss: 1.4938 - accuracy: 0.8528 - val_loss: 1.1001 - val_accuracy: 0.8750
Epoch 4/10
720/720 [==============================] - 0s 623us/sample - loss: 0.7229 - accuracy: 0.9278 - val_loss: 0.2779 - val_accuracy: 0.9250
Epoch 5/10
720/720 [==============================] - 0s 624us/sample - loss: 0.3080 - accuracy: 0.9625 - val_loss: 0.7372 - val_accuracy: 0.8750
Epoch 6/10
720/720 [==============================] - 0s 616us/sample - loss: 0.2678 - accuracy: 0.9653 - val_loss: 0.1045 - val_accuracy: 0.9750
Epoch 7/10
720/720 [==============================] - 0s 584us/sample - loss: 0.1

In [ ]:
test_features, test_labels = preprocess_data(test, spec_shape=(26, 8))

In [ ]:
metrics = model.evaluate()